In [1]:
import pandas as pd
import numpy as np
import re

from gensim.models.fasttext import FastText
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

RAND = 10

In [2]:
train_supervised_path = 'data/train_supervised_dataset.csv'
train_unsupervised_path = 'data/train_unsupervised_dataset.csv'
test_path = 'data/test_dataset.csv'

In [3]:
df_sup = pd.read_csv(train_supervised_path).fillna("")

In [4]:
df_unsup = pd.read_csv(train_unsupervised_path).fillna("")

In [5]:
df_test = pd.read_csv(test_path)

In [6]:
# Функция очистки текста
def clean_text(text):
    text = text.lower()
    #text = re.sub(r'[^\sa-zA-Z0-9@\[\]]',' ',text) # Удаляет пунктцацию
    text = re.sub(r'\w*\d+\w*', '', text) # Удаляет цифры
    text = re.sub(r'[^\w\s]', ' ', text) # Удаляет знаки
    text = re.sub(r'\b\S{1}\b', '', text) # Удаляет слова из 1-й буквы
    text = re.sub(r'\b\S{2}\b', '', text) # Удаляет слова из 2-х букв
    text = re.sub('\s{2,}', " ", text) # Удаляет ненужные пробелы
    return text

# Применяем ее к тексту
df_sup['name'] = df_sup['name'].apply(clean_text)
df_unsup['name'] = df_unsup['name'].apply(clean_text)
df_test['name'] = df_test['name'].apply(clean_text)

In [7]:
#print(f"Доля пропусков в good: {round(df_sup['good'].isna().sum()/df_sup.shape[0]*100, 3)}%")
#print(f"Доля пропусков в brand: {round(df_sup['brand'].isna().sum()/df_sup.shape[0]*100, 3)}%")

In [8]:
# заполняет пропуски
#for i in df_sup.columns:
#    if i == 'good':
#        df_sup['good'].fillna('None', inplace=True)
#    elif i == 'brand':
#        df_sup['brand'].fillna('None', inplace=True)
        
#df_sup.isna().sum()   

In [9]:
df_sup['tokens'] = df_sup['name'].str.lower().str.split()
df_unsup['tokens'] = df_unsup['name'].str.lower().str.split()
df_test["tokens"] = df_test["name"].str.lower().str.split()

In [10]:
def apply_bio_tagging(row):
    """
    По токенам чека и разметке (то есть выделенным товарам и брендам) строим BIO-теги
    """
    tokens = row["tokens"]
    good = row["good"].split(',')[0].split()
    brand = row["brand"].split(',')[0].split()
    tags = ['O'] * len(tokens)
    for i, token in enumerate(tokens):
        if len(good) > 0 and tokens[i:i + len(good)] == good:
            tags[i] = "B-GOOD"
            for j in range(i + 1, i + len(good)):
                tags[j] = "I-GOOD"
        if len(brand) > 0 and tokens[i:i + len(brand)] == brand:
            tags[i] = "B-BRAND"
            for j in range(i + 1, i + len(brand)):
                tags[j] = "I-BRAND"
    return tags

In [11]:
df_sup["tags"] = df_sup.apply(apply_bio_tagging, axis=1)

In [12]:
df_sup

,id,name,good,brand,tokens,tags
0,0,petmax бантик леопард красн розой,бантик,petmax,"[petmax, бантик, леопард, красн, розой]","[B-BRAND, B-GOOD, O, O, O]"
1,1,бусы для елки,бусы,,"[бусы, для, елки]","[B-GOOD, O, O]"
2,2,футболка piazza italia,футболка,piazza italia,"[футболка, piazza, italia]","[B-GOOD, B-BRAND, I-BRAND]"
3,3,one заколка для волос для девочки,заколка,,"[one, заколка, для, волос, для, девочки]","[O, B-GOOD, O, O, O, O]"
4,4,одежда вес,одежда,,"[одежда, вес]","[B-GOOD, O]"
...,...,...,...,...,...,...
24995,24995,вода саирме,вода,sairme,"[вода, саирме]","[B-GOOD, O]"
24996,24996,моя семя ассортим,,моя семья,"[моя, семя, ассортим]","[O, O, O]"
24997,24997,рулет бисквитн яшкино клубничный слив,рулет,яшкино,"[рулет, бисквитн, яшкино, клубничный, слив]","[B-GOOD, O, B-BRAND, O, O]"
24998,24998,почвогрунт цветочное счастье фаско декоративн...,почвогрунт,фаско,"[почвогрунт, цветочное, счастье, фаско, декора...","[B-GOOD, O, O, B-BRAND, O, O, O]"


In [13]:
df_unsup

,id,name,tokens
0,0,зубная щетка орал три эффект деликатное отбели...,"[зубная, щетка, орал, три, эффект, деликатное,..."
1,1,салфетки vister влажные для,"[салфетки, vister, влажные, для]"
2,2,платье женское светло серый,"[платье, женское, светло, серый]"
3,3,лакомство деревенские лакомства собак мини пор...,"[лакомство, деревенские, лакомства, собак, мин..."
4,4,суппорт гитарный ergo play troster,"[суппорт, гитарный, ergo, play, troster]"
...,...,...,...
999995,999995,спонж макияжа фигурный,"[спонж, макияжа, фигурный]"
999996,999996,дифф arom,"[дифф, arom]"
999997,999997,матрас надувной tropical bird запл ремонта диз...,"[матрас, надувной, tropical, bird, запл, ремон..."
999998,999998,пододеяльник стм страйп размер страйп сатин хл...,"[пододеяльник, стм, страйп, размер, страйп, са..."


In [14]:
fst_model = FastText(df_unsup['tokens'],
                     vector_size=300,
                     window=3,
                     min_count=1,
                     sg=1,
                     alpha=0.1,
                     negative=10,
                     epochs=10)

In [ ]:
fst_model.save('fst.model')

In [ ]:
def to_series(column):
    token_list = column.to_list()
    flat_list = [item for sublist in token_list for item in sublist]
    tokens_series = pd.Series(flat_list)
    return tokens_series

In [ ]:
def make_datasets(data):
    tags = to_series(df_sup["tags"])
    tokenss = to_series(df_sup["tokens"])
    dataset = tokenss.apply(lambda x: fst_model.wv[x])
    dtv_train = pd.concat([pd.DataFrame(dataset.to_list()), 
           pd.DataFrame(tags)], axis=1, ignore_index=True)
    dtv_train.rename(columns={300:"labels"}, inplace=True)

    good_train = dtv_train.loc[dtv_train['labels'].isin(["B-GOOD", "I-GOOD", "O"])]
    brand_train = dtv_train.loc[dtv_train['labels'].isin(["B-BRAND", "I-BRAND", "O"])]
    
    
    return good_train, brand_train

In [ ]:
good_train, brand_train = make_datasets(df_sup)

In [ ]:
X_good = good_train.drop("labels", axis=1)
y_good = good_train.labels

X_brand = brand_train.drop("labels", axis=1)
y_brand = brand_train.labels

X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_good,
                                                            y_good,
                                                            test_size=0.2,
                                                            shuffle=True,
                                                            random_state=RAND)
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_brand,
                                                            y_brand,
                                                            test_size=0.2,
                                                            shuffle=True,
                                                            random_state=RAND)

eval_set_g = [(X_test_g, y_test_g)]
eval_set_b = [(X_test_b, y_test_b)]

In [ ]:
clf_g = CatBoostClassifier(random_state=RAND,
                           eval_metric='TotalF1',
                           loss_function='MultiClass')
clf_b = CatBoostClassifier(random_state=RAND,
                           eval_metric='TotalF1',
                           loss_function='MultiClass')

clf_g.fit(X_train_g,
          y_train_g,
          eval_set=eval_set_g,
          early_stopping_rounds=100, 
          use_best_model=True,
          verbose=False)

clf_b.fit(X_train_b,
          y_train_b,
          eval_set=eval_set_b,
          early_stopping_rounds=100, 
          use_best_model=True,
          verbose=False)

In [ ]:
def create_submit(row):
    good = []

    for i in row:
        try:
            word_array = fst_model.wv[i]
            dataset = pd.DataFrame(word_array.reshape((1, 300)))
            pred = clf_g.predict(dataset)
            if pred[0][0] == "B-GOOD":
                good.append(i)
        except Exception as ex:
            pass
    
    return ''.join(good)

In [ ]:
def create_submit_brand(row):
    brand = []

    for i in row:
        try:
            word_array = fst_model.wv[i]
            dataset = pd.DataFrame(word_array.reshape((1, 300)))
            pred = clf_b.predict(dataset)
            if pred[0][0] == "B-BRAND":
                brand.append(i)
        except Exception as ex:
            pass
    
    return ''.join(brand)

In [ ]:
df_test["good"] = df_test["tokens"].apply(create_submit)
df_test["brand"] = df_test["tokens"].apply(create_submit_brand)

In [ ]:
df_fin = df_test.drop(['name', 'tokens'], axis=1, inplace=True)

In [ ]:
df_test.to_csv('itog.csv')

In [ ]:
grid = {
    'n_estimators': list(range(1000, 5000, 300)),
    #'learning_rate': np.linspace(0.01, 0.1, 5),
    #'boosting_type' : ['Ordered', 'Plain'],
    #'max_depth': list(range(3, 12)),
    #'l2_leaf_reg': np.logspace(-5, 2, 5),
    #'random_strength': list(range(10, 50, 5)),
    #'bootstrap_type': ["Bayesian", "Bernoulli", "MVS", "No"],
    #'border_count': [128, 254],
    #'grow_policy': ["SymmetricTree", "Depthwise", "Lossguide"],
    'random_state': [RAND]
}

model_g = CatBoostClassifier(random_state=RAND,
                             eval_metric='TotalF1',
                             loss_function='MultiClass',
                             silent=True)
grid_search_result = model_g.randomized_search(grid,
                                              X=X_train_g,
                                              y=y_train_g, 
                                              verbose=False)